## Valorant Decision Tree Model

In [48]:
# Initial imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping
from sqlalchemy import create_engine, Column, Integer, String, Boolean, Float, ForeignKey
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


## Loading and Preparing Agent Data

In [49]:
import sqlite3

# Create database connection
engine = create_engine('sqlite:///SQLite/Valorant.sqlite')

In [50]:
# Querying and creating Dataframe
query_1 = """
    SELECT agents.agent, round_results.roundResult, player_stats.competitiveTier, player_stats.assists, player_stats.kills, player_stats.deaths, team_stats.won
    FROM agents
    INNER JOIN round_results ON agents.id = round_results.id
    INNER JOIN player_stats ON agents.id = player_stats.id
    INNER JOIN team_stats ON agents.id = team_stats.id 
"""

agent_df = pd.read_sql(query_1, engine)
agent_df

,agent,roundResult,competitiveTier,assists,kills,deaths,won
0,Gekko,Bomb detonated,IRON_1,0,4,8,0
1,Fade,Eliminated,IRON_2,3,22,16,1
2,Breach,Eliminated,UNRANKED,2,12,17,1
3,Deadlock,Eliminated,IRON_2,5,17,13,0
4,Raze,Eliminated,IRON_2,4,9,11,0
5,Chamber,Bomb detonated,IRON_3,10,10,10,1
6,KAY/O,Eliminated,IRON_3,3,4,18,1
7,Skye,Bomb defused,IRON_3,4,27,12,0
8,Cypher,Bomb defused,IRON_1,3,12,10,1
9,Sova,Eliminated,IRON_3,3,14,16,0


In [51]:
from sklearn.preprocessing import LabelEncoder

# Initializing LabelEncoder for roundResult and competitiveTier
label_encoder1 = LabelEncoder()
label_encoder2 = LabelEncoder()

# Applying label encoding
agent_df["roundResult_encoded"] = label_encoder1.fit_transform(agent_df["roundResult"])
agent_df["competitiveTier_encoded"] = label_encoder2.fit_transform(agent_df["competitiveTier"])

# Dropping and rearranging columns
columns_to_drop = ["roundResult", "competitiveTier"]
agent_df_refined = agent_df.drop(columns=columns_to_drop)

agent_df_refined

,agent,assists,kills,deaths,won,roundResult_encoded,competitiveTier_encoded
0,Gekko,0,4,8,0,1,5
1,Fade,3,22,16,1,2,6
2,Breach,2,12,17,1,2,10
3,Deadlock,5,17,13,0,2,6
4,Raze,4,9,11,0,2,6
5,Chamber,10,10,10,1,1,7
6,KAY/O,3,4,18,1,2,7
7,Skye,4,27,12,0,0,7
8,Cypher,3,12,10,1,0,5
9,Sova,3,14,16,0,2,7


In [52]:
# Isolating our independent variable X
X = agent_df_refined.drop("agent", axis=1).values
X[:5]

array([[ 0,  4,  8,  0,  1,  5],
       [ 3, 22, 16,  1,  2,  6],
       [ 2, 12, 17,  1,  2, 10],
       [ 5, 17, 13,  0,  2,  6],
       [ 4,  9, 11,  0,  2,  6]])

In [53]:
# Isolating our dependent variable y
y = agent_df_refined["agent"].values
y

array(['Gekko', 'Fade', 'Breach', 'Deadlock', 'Raze', 'Chamber', 'KAY/O',
       'Skye', 'Cypher', 'Sova', 'Sova', 'Killjoy', 'Harbor', 'Viper',
       'Phoenix', 'Astra', 'Brimstone', 'Neon', 'Yoru', 'Sage', 'Reyna',
       'Omen', 'Jett', 'Gekko', 'Fade', 'Breach', 'Deadlock', 'Raze',
       'Chamber', 'KAY/O', 'Skye', 'Cypher', 'Sova', 'Sova', 'Killjoy',
       'Harbor', 'Viper', 'Phoenix', 'Astra', 'Brimstone', 'Neon', 'Yoru',
       'Sage', 'Reyna', 'Omen', 'Jett'], dtype=object)

# Finding a model and compiling, training and evaluating said model

In [54]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_test_scaled.shape

(12, 6)

In [55]:
# building tuner to find best model


input_dim = 43

# Define a model-building function for Keras Tuner
def build_model(hp):
    model = tf.keras.models.Sequential()

    # Input layer with the specified number of features
    model.add(layers.Input(shape=(input_dim,)))

    # Add hidden layers with tunable units
    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
        model.add(tf.keras.layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                                            activation=hp.Choice(f'activation_{i}', values=['relu', 'tanh'])))

    # Output layer for binary classification
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Define a Keras Tuner instance
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=30,
    directory='tuner_results',
    )

# Setting up the early stop variable for faster tests
early_stopping = EarlyStopping(
    monitor='val_loss',    
    patience=5,            
    restore_best_weights=True)

## Building the Neural Network (work in progress)

In [ ]:
# Create the Keras Sequential model
agent_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
agent_model.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=2))

# Add the output layer that uses a probability activation function
agent_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
agent_model.summary()

In [ ]:
# Compile the Sequential model together and customize metrics
agent_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
 # Fit the model to the training data
fit_model = agent_model.fit(X_train_scaled, y_train, epochs=50)

In [ ]:
 # Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset.
model_loss, model_accuracy = agent_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")